# SAE-70 - Nettoyage Texte Basique

**En tant que** NLP engineer  
**Je veux** nettoyer le texte preprocessé  
**Afin de** le préparer pour l'analyse NLP

## ⚠️ Prérequis

- SAE-97 doit être exécuté (Nettoyage Données Reviews)
- Fichier source: `data/cleaned/reviews_clean.parquet`

## Critères d'acceptation

- [x] Charger reviews_clean.parquet
- [x] Lowercase (tout en minuscules)
- [x] Suppression ponctuation
- [x] Suppression URLs
- [x] Suppression emails
- [x] Suppression chiffres (optionnel, selon besoin)
- [x] Suppression espaces multiples
- [x] Fonction réutilisable créée
- [x] Colonne `text_cleaned` ajoutée au DataFrame

---
## 1. Imports et Configuration

In [ ]:
import re
import pandas as pd
from pathlib import Path

# Configuration des chemins
DATA_PATH = Path('../data/cleaned')
INPUT_FILE = DATA_PATH / 'reviews_clean.parquet'
OUTPUT_FILE = DATA_PATH / 'reviews_text_cleaned.parquet'

print(f"📁 Fichier source: {INPUT_FILE}")
print(f"📁 Fichier destination: {OUTPUT_FILE}")

---
## 2. Chargement des données

In [ ]:
# Vérification que le fichier source existe
if not INPUT_FILE.exists():
    raise FileNotFoundError(
        f"❌ Le fichier {INPUT_FILE} n'existe pas.\n"
        f"⚠️ Exécutez d'abord SAE-97 (Nettoyage Données Reviews)"
    )

# Chargement des données
df_reviews = pd.read_parquet(INPUT_FILE)

print(f"✅ Données chargées: {len(df_reviews):,} reviews")
print(f"📊 Colonnes: {list(df_reviews.columns)}")
df_reviews.head()

In [ ]:
# Vérification de la colonne text
if 'text' not in df_reviews.columns:
    raise KeyError("❌ La colonne 'text' n'existe pas dans le DataFrame")

# Aperçu du texte avant nettoyage
print("📝 Exemples de textes avant nettoyage:")
print("="*50)
for i, text in enumerate(df_reviews['text'].head(3).values):
    preview = text[:200] + "..." if len(str(text)) > 200 else text
    print(f"\n[{i+1}] {preview}")

---
## 3. Fonction de Nettoyage Texte

In [ ]:
def clean_text(text: str, remove_digits: bool = False) -> str:
    """
    Nettoie le texte pour l'analyse NLP.
    
    Opérations effectuées:
    - Conversion en minuscules
    - Suppression des URLs
    - Suppression des emails
    - Suppression de la ponctuation
    - Suppression des chiffres (optionnel)
    - Suppression des espaces multiples
    
    Parameters
    ----------
    text : str
        Le texte à nettoyer
    remove_digits : bool, default=False
        Si True, supprime également les chiffres
    
    Returns
    -------
    str
        Le texte nettoyé
        
    Examples
    --------
    >>> clean_text("The food was GREAT!!! 😋 Visit http://example.com")
    'the food was great visit'
    
    >>> clean_text("Price: $25.99", remove_digits=True)
    'price'
    """
    # Gestion des valeurs manquantes
    if pd.isna(text) or text is None:
        return ""
    
    # Conversion en string si nécessaire
    text = str(text)
    
    # 1. Conversion en minuscules
    text = text.lower()
    
    # 2. Suppression des URLs
    text = re.sub(r'http\S+|www\.\S+|https\S+', '', text)
    
    # 3. Suppression des emails
    text = re.sub(r'\S+@\S+', '', text)
    
    # 4. Suppression de la ponctuation (garde lettres, chiffres, espaces)
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # 5. Suppression des chiffres (optionnel)
    if remove_digits:
        text = re.sub(r'\d+', '', text)
    
    # 6. Suppression des espaces multiples et trim
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

print("✅ Fonction clean_text() définie")

### 3.1 Tests de la fonction

In [ ]:
# Tests de la fonction de nettoyage
test_cases = [
    ("The food was GREAT!!! 😋", "the food was great"),
    ("Visit http://example.com for more!", "visit for more"),
    ("Contact: test@email.com", "contact"),
    ("Price: $25.99 only!", "price 25 99 only"),
    ("   Multiple   spaces   here   ", "multiple spaces here"),
    (None, ""),
    ("", ""),
]

print("🧪 Tests de la fonction clean_text():")
print("="*60)

all_passed = True
for i, (input_text, expected) in enumerate(test_cases, 1):
    result = clean_text(input_text)
    status = "✅" if result == expected else "❌"
    if result != expected:
        all_passed = False
    print(f"{status} Test {i}:")
    print(f"   Input:    {repr(input_text)}")
    print(f"   Expected: {repr(expected)}")
    print(f"   Got:      {repr(result)}")
    print()

if all_passed:
    print("\n🎉 Tous les tests sont passés!")
else:
    print("\n⚠️ Certains tests ont échoué")

---
## 4. Application du Nettoyage

In [ ]:
%%time

# Application de la fonction de nettoyage
print("🔄 Nettoyage du texte en cours...")

# Création de la colonne text_cleaned
df_reviews['text_cleaned'] = df_reviews['text'].apply(clean_text)

print(f"✅ Nettoyage terminé pour {len(df_reviews):,} reviews")

### 4.1 Comparaison Avant/Après

In [ ]:
# Comparaison avant/après
print("📊 Comparaison Avant/Après Nettoyage:")
print("="*70)

for i in range(min(5, len(df_reviews))):
    original = str(df_reviews['text'].iloc[i])[:150]
    cleaned = str(df_reviews['text_cleaned'].iloc[i])[:150]
    
    print(f"\n🔹 Review {i+1}:")
    print(f"   AVANT:  {original}...")
    print(f"   APRÈS:  {cleaned}...")

---
## 5. Statistiques du Nettoyage

In [ ]:
# Statistiques sur le nettoyage
print("📈 Statistiques du Nettoyage:")
print("="*50)

# Longueur moyenne avant/après
avg_len_before = df_reviews['text'].str.len().mean()
avg_len_after = df_reviews['text_cleaned'].str.len().mean()
reduction = ((avg_len_before - avg_len_after) / avg_len_before) * 100

print(f"\n📏 Longueur moyenne:")
print(f"   Avant:  {avg_len_before:.0f} caractères")
print(f"   Après:  {avg_len_after:.0f} caractères")
print(f"   Réduction: {reduction:.1f}%")

# Textes vides après nettoyage
empty_count = (df_reviews['text_cleaned'] == '').sum()
empty_pct = (empty_count / len(df_reviews)) * 100

print(f"\n🔍 Textes vides après nettoyage:")
print(f"   Count: {empty_count:,} ({empty_pct:.2f}%)")

# Nombre de mots moyen
avg_words_before = df_reviews['text'].str.split().str.len().mean()
avg_words_after = df_reviews['text_cleaned'].str.split().str.len().mean()

print(f"\n📝 Nombre de mots moyen:")
print(f"   Avant:  {avg_words_before:.1f} mots")
print(f"   Après:  {avg_words_after:.1f} mots")

---
## 6. Sauvegarde des Données

In [ ]:
# Sauvegarde du DataFrame avec la nouvelle colonne
print("💾 Sauvegarde des données...")

# Option 1: Nouveau fichier (recommandé pour traçabilité)
df_reviews.to_parquet(OUTPUT_FILE, index=False)
print(f"✅ Sauvegardé: {OUTPUT_FILE}")

# Option 2: Mise à jour du fichier original (décommenter si souhaité)
# df_reviews.to_parquet(INPUT_FILE, index=False)
# print(f"✅ Fichier original mis à jour: {INPUT_FILE}")

# Vérification
file_size = OUTPUT_FILE.stat().st_size / (1024 * 1024)
print(f"📊 Taille du fichier: {file_size:.2f} MB")

---
## 7. Vérification Finale

In [ ]:
# Vérification de la sauvegarde
df_check = pd.read_parquet(OUTPUT_FILE)

print("✅ Vérification finale:")
print(f"   - Nombre de reviews: {len(df_check):,}")
print(f"   - Colonnes: {list(df_check.columns)}")
print(f"   - Colonne text_cleaned présente: {'text_cleaned' in df_check.columns}")

df_check[['text', 'text_cleaned']].head()

---
## ✅ Résumé SAE-70

### Critères d'acceptation validés:

| Critère | Status |
|---------|--------|
| Charger reviews_clean.parquet | ✅ |
| Lowercase | ✅ |
| Suppression ponctuation | ✅ |
| Suppression URLs | ✅ |
| Suppression emails | ✅ |
| Suppression chiffres (optionnel) | ✅ |
| Suppression espaces multiples | ✅ |
| Fonction réutilisable créée | ✅ |
| Colonne text_cleaned ajoutée | ✅ |

### Fichiers:
- **Input**: `data/cleaned/reviews_clean.parquet`
- **Output**: `data/cleaned/reviews_text_cleaned.parquet`